In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 64
epoch = 1000/10

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
FGSM = []
PGD_10 = []
PGD_100 = []
for base_model_layer in range(0, 51, 10):
    FGSM.append(onp.load("./npy/%s_FGSM_%d_layer.npy"%(DATASET, base_model_layer)))
    PGD_10.append(onp.load("./npy/%s_PGD_10_%d_layer.npy"%(DATASET, base_model_layer)))
    PGD_100.append(onp.load("./npy/%s_PGD_10_%d_layer.npy"%(DATASET, base_model_layer)))

In [6]:
y_test = onp.load("./npy/%s_y_test.npy"%(DATASET))

In [7]:
x_test = onp.load("./npy/%s_x_test.npy"%(DATASET))

In [8]:
class Model(tf.keras.Model):
    def __init__(self, layers):
        super(Model, self).__init__()
        self.layers_num = layers
        self.conv = [tf.keras.layers.Conv2D(64, (3,3)) for i in range(self.layers_num)]
        self.flatten = tf.keras.layers.Flatten()
        self.out = tf.keras.layers.Dense(10)
        
    def call(self, inputs):
        x = inputs
        for f in self.conv:
            x = f(x)
            x = tf.math.erf(x)
            
        x = self.flatten(x)
        out = self.out(x)
        return out

In [9]:
model = Model(9)
model.compile(optimizer='adam',
          loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

In [10]:
x_train_all, x_test_all = x_train_all.reshape(-1, 32, 32, 3), x_test_all.reshape(-1, 32, 32, 3)

In [13]:
model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=20, validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 4s 97us/sample - loss: 1.4938 - accuracy: 0.6013 - val_loss: 2.4261 - val_accuracy: 0.4790
Epoch 2/20
45000/45000 [==============================] - 4s 96us/sample - loss: 1.5111 - accuracy: 0.6028 - val_loss: 2.3497 - val_accuracy: 0.4944
Epoch 3/20
45000/45000 [==============================] - 4s 97us/sample - loss: 1.4862 - accuracy: 0.6136 - val_loss: 2.5096 - val_accuracy: 0.4716
Epoch 4/20
45000/45000 [==============================] - 4s 97us/sample - loss: 1.4725 - accuracy: 0.6208 - val_loss: 2.5305 - val_accuracy: 0.4850
Epoch 5/20
45000/45000 [==============================] - 4s 96us/sample - loss: 1.4780 - accuracy: 0.6256 - val_loss: 2.7737 - val_accuracy: 0.4780
Epoch 6/20
45000/45000 [==============================] - 4s 95us/sample - loss: 1.5053 - accuracy: 0.6263 - val_loss: 2.7730 - val_accuracy: 0.4662
Epoch 7/20
45000/45000 [==============================] -

In [12]:
model.evaluate(x_test.reshape(-1, 32, 32, 3), y_test, verbose=0)

[2.493234734982252, 0.46142578]

In [ ]:
model.evaluate(FGSM[2].reshape(-1, 32, 32, 3), y_test, verbose=0)

In [ ]:
loss_curve = []
acc_curve  = []

loss_curve_FGSM = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_FGSM  = {idx*10+1: [] for idx in range(len(FGSM))}

loss_curve_PGD_10 = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_PGD_10  = {idx*10+1: [] for idx in range(len(FGSM))}

loss_curve_PGD_100 = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_PGD_100  = {idx*10+1: [] for idx in range(len(FGSM))}

for i in range(run):
    model = Model(10)
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    for j in range(int(epoch)):
        model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=10)
        # CLEAN
        r = model.evaluate(x_test, y_test, verbose=0)
        loss_curve.append(r[0])
        acc_curve.append(r[1])
        
        for idx, (F, _10, _100) in enumerate(zip(FGSM, PGD_10, PGD_100)):
                
            r = model.evaluate(F,    y_test, verbose=0)
            print("FGSM    %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_FGSM[idx*10+1].append(r[0])
            acc_curve_FGSM[idx*10+1].append(r[1])
        
            r = model.evaluate(_10,  y_test, verbose=0)
            print("PGD-10  %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_PGD_10[idx*10+1].append(r[0])
            acc_curve_PGD_10[idx*10+1].append(r[1])
            
            r = model.evaluate(_100, y_test, verbose=0)
            print("PGD-100 %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_PGD_100[idx*10+1].append(r[0])
            acc_curve_PGD_100[idx*10+1].append(r[1])
            

In [ ]:
loss_curve_FGSM

In [ ]:
for idx, (F, _10, _100) in enumerate(zip(FGSM, PGD_10, PGD_100)):
    r = model.evaluate(F,    y_test, verbose=0)
    print("FGSM    %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
    r = model.evaluate(_10,  y_test, verbose=0)
    print("PGD-10  %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
    r = model.evaluate(_100, y_test, verbose=0)
    print("PGD-100 %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))